<a href="https://colab.research.google.com/github/sirlapat/ML_Assignment1/blob/main/ML_AS_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00
time: 440 µs (started: 2023-12-01 22:49:30 +00:00)


In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [5]:
torch.manual_seed(123)

In [6]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

100%|██████████| 170498071/170498071 [00:02<00:00, 80160044.31it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [7]:
mean

tensor([0.4914, 0.4822, 0.4465])

In [8]:
std

tensor([0.2470, 0.2435, 0.2616])

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
device

device(type='cpu')

In [11]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
])

In [12]:

cifar10 = datasets.CIFAR10(
    './data', train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [13]:
cifar10_val = datasets.CIFAR10(
     './data', train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [14]:
first_image, label = cifar10[0]
print(first_image.shape)

torch.Size([3, 32, 32])


In [15]:
train_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)

In [16]:
# Assuming 'device' is defined previously
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model definition
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [17]:

def train_model(model, train_loader, test_loader, num_epochs=300, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Testing the model
        model.eval()
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = correct / total

        if epoch % 1 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Test Accuracy: {accuracy * 100:.2f}%')

    # Classification Report
    report = classification_report(all_labels, all_predicted)
    print("Classification Report:\n", report)

In [ ]:
train_model(model, train_loader, test_loader, num_epochs=300, lr=0.01)

Epoch 1/300, Loss: 1.7886348081870638, Test Accuracy: 40.94%
Epoch 2/300, Loss: 1.6521172705020037, Test Accuracy: 43.15%
Epoch 3/300, Loss: 1.5804378465437692, Test Accuracy: 44.45%
Epoch 4/300, Loss: 1.5198366514246813, Test Accuracy: 45.26%
Epoch 5/300, Loss: 1.4625313013544161, Test Accuracy: 46.44%
Epoch 6/300, Loss: 1.4085316710646, Test Accuracy: 47.50%
Epoch 7/300, Loss: 1.3539841107580803, Test Accuracy: 47.11%
Epoch 8/300, Loss: 1.3031534532744078, Test Accuracy: 47.23%
Epoch 9/300, Loss: 1.249340126015632, Test Accuracy: 48.36%
Epoch 10/300, Loss: 1.1990976107097633, Test Accuracy: 47.90%
Epoch 11/300, Loss: 1.147438968516891, Test Accuracy: 48.39%
Epoch 12/300, Loss: 1.0958708692496966, Test Accuracy: 49.10%
Epoch 13/300, Loss: 1.0460041413990564, Test Accuracy: 48.73%
Epoch 14/300, Loss: 0.9943198018247931, Test Accuracy: 48.12%
Epoch 15/300, Loss: 0.9466876947223873, Test Accuracy: 48.06%
Epoch 16/300, Loss: 0.8974496146195681, Test Accuracy: 48.03%
Epoch 17/300, Loss: 0.

In [ ]:
input_size = 32 * 32 * 3
hidden_sizes = [512, 256, 128]
output_size = 10

layers = []
layers.append(nn.Flatten())

# Adding hidden layers with Tanh activation function
for i in range(len(hidden_sizes)):
    layers.append(nn.Linear(input_size if i == 0 else hidden_sizes[i - 1], hidden_sizes[i]))
    layers.append(nn.Tanh())

# Output layer
layers.append(nn.Linear(hidden_sizes[-1], output_size))

model2 = nn.Sequential(*layers).to(device)

time: 20.1 ms (started: 2023-12-02 01:36:27 +00:00)


In [ ]:
model2 = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 256),
    nn.Tanh(),
    nn.Linear(256, 128),
    nn.Tanh(),
    nn.Linear(128, 10)
).to(device)

time: 18.1 ms (started: 2023-12-02 01:36:32 +00:00)


In [ ]:
train_model(model2, train_loader, test_loader, num_epochs=300, lr=0.02)

Epoch 1/300, Loss: 1.5464903359526025, Test Accuracy: 45.85%
Epoch 2/300, Loss: 1.4621418760285039, Test Accuracy: 46.92%
Epoch 3/300, Loss: 1.389218071905833, Test Accuracy: 47.25%
Epoch 4/300, Loss: 1.3159023884466956, Test Accuracy: 46.78%
Epoch 5/300, Loss: 1.242584570698912, Test Accuracy: 47.96%
Epoch 6/300, Loss: 1.16827581127866, Test Accuracy: 47.50%
Epoch 7/300, Loss: 1.0966241708262487, Test Accuracy: 48.47%
Epoch 8/300, Loss: 1.0286566393923011, Test Accuracy: 48.42%
Epoch 9/300, Loss: 0.9540793951207526, Test Accuracy: 46.65%
Epoch 10/300, Loss: 0.8861436463668418, Test Accuracy: 48.49%
Epoch 11/300, Loss: 0.8212438619106303, Test Accuracy: 47.75%
Epoch 12/300, Loss: 0.7507310081809587, Test Accuracy: 46.75%
Epoch 13/300, Loss: 0.6914542057692662, Test Accuracy: 46.30%
Epoch 14/300, Loss: 0.6358378670837966, Test Accuracy: 47.92%
Epoch 15/300, Loss: 0.5780094443741168, Test Accuracy: 47.44%
Epoch 16/300, Loss: 0.5192055108927796, Test Accuracy: 47.40%
Epoch 17/300, Loss: 0